# Proficiency Test

#### NAME : RAMZY MOHAMMAD
#### IYKRA DATA FELLOWSHIP BATCH 6

In [218]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

In [219]:
df = pd.read_csv("mantap.csv")

c:\users\asus\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [220]:
df

,INCIDENT_NUMBER,DISTRICT,YEAR,MONTH,DAY_OF_WEEK,HOUR,district,offense_description
0,I192077559,B3,2018,4,Monday,9,B,investigate person
1,I192077332,E18,2018,3,Tuesday,8,E,larceny all others
2,I192076660,E5,2018,10,Wednesday,12,E,harassment
3,I192075386,A1,2018,4,Monday,8,A,harassment
4,I192075335,D4,2018,1,Monday,0,D,property
...,...,...,...,...,...,...,...,...
189078,20063425,A7,2020,9,Tuesday,0,A,investigate property
189079,20062356,E18,2020,8,Friday,18,E,fraud
189080,20046400,B2,2020,7,Thursday,1,B,weapon violation
189081,20038446,B2,2020,6,Wednesday,1,B,weapon violation


In [221]:
df.isna().sum()

INCIDENT_NUMBER        0
DISTRICT               0
YEAR                   0
MONTH                  0
DAY_OF_WEEK            0
HOUR                   0
district               0
offense_description    0
dtype: int64

In [222]:
df = df.sample(n=7000)

In [223]:
df

,INCIDENT_NUMBER,DISTRICT,YEAR,MONTH,DAY_OF_WEEK,HOUR,district,offense_description
158446,202043007,E18,2020,6,Friday,12,E,larceny theft from mv
51903,I182050041,B3,2018,6,Tuesday,13,B,assault simple
138134,202070413,A7,2020,9,Saturday,0,A,sick
50792,I182051164,B2,2018,6,Friday,20,B,death investigation
149590,202055011,B2,2020,8,Sunday,20,B,motor vehicle accident
...,...,...,...,...,...,...,...,...
155549,202046935,A7,2020,7,Friday,21,A,investigate property
149029,202055753,A15,2020,8,Wednesday,12,A,investigate person
85500,I182014288,C11,2018,2,Friday,17,C,assault simple
70776,I182030023,B2,2018,4,Sunday,8,B,sick


In [224]:
df = df.replace(r'^\s*$', np.NaN, regex=True)

# DataSplitting

In [225]:
X = df.drop(columns=["INCIDENT_NUMBER", "YEAR", "HOUR", "district"])
y = df.HOUR

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5600, 4), (1400, 4), (5600,), (1400,))

# Training

In [226]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5600 entries, 2441 to 73137
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   DISTRICT             5600 non-null   object
 1   MONTH                5600 non-null   int64 
 2   DAY_OF_WEEK          5600 non-null   object
 3   offense_description  5600 non-null   object
dtypes: int64(1), object(3)
memory usage: 218.8+ KB


In [227]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

# Modelling

## RandomForestRegressor

In [248]:
gsp.rf_params

{'algo__n_estimators': [100, 150, 200],
 'algo__max_depth': [20, 50, 80],
 'algo__max_features': [0.3, 0.6, 0.8],
 'algo__min_samples_leaf': [1, 5, 10]}

In [249]:
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(), ['MONTH']),
    ('categoric', cat_pipe(encoder='onehot'), ['DAY_OF_WEEK', 'DISTRICT', 'offense_description']),
    
])


pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', RandomForestRegressor(n_jobs=-1, random_state=42))
])


model = GridSearchCV(pipeline, gsp.rf_params, scoring = 'r2', cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

print(model.best_params_)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:  4.1min finished


{'algo__max_depth': 20, 'algo__max_features': 0.3, 'algo__min_samples_leaf': 10, 'algo__n_estimators': 200}
0.08713449086717528 0.006529914377437773 0.018585032412645086


## SVMRegressor

In [230]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from jcopml.tuning import grid_search_params as gsp

In [231]:
gsp.svm_params

{'algo__gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
 'algo__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])}

In [233]:
preprocessor = ColumnTransformer([
    ('categoric1', cat_pipe(encoder='ordinal'), ['MONTH']),
    ('categoric', cat_pipe(encoder='onehot'), ['DAY_OF_WEEK', 'DISTRICT', 'offense_description']),
])


pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', SVR(max_iter=500))
])

model = GridSearchCV(pipeline, gsp.svm_params, cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

print(model.best_params_)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

Fitting 3 folds for each of 49 candidates, totalling 147 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 147 out of 147 | elapsed:   22.6s finished
c:\users\asus\appdata\local\programs\python\python38\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


{'algo__C': 0.001, 'algo__gamma': 10.0}
-0.0960835799848867 -0.05390702147721297 -0.08794601517976242


## KNNRegressor

In [234]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [235]:
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

In [236]:
from sklearn.compose import ColumnTransformer

In [237]:
preprocessor = ColumnTransformer([
    ('numeric', numerical_pipeline, ['MONTH']),
    ('categoric', categorical_pipeline, ['DAY_OF_WEEK', 'DISTRICT','offense_description'])
])

In [238]:
from sklearn.neighbors import KNeighborsRegressor

In [239]:
pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', KNeighborsRegressor())
])

In [240]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['MONTH']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['DAY_OF_WEEK', 'DISTRICT',
           

In [241]:
pipeline.score(X_test, y_test)

-0.15428206711006642

In [244]:
from sklearn.model_selection import GridSearchCV

In [245]:
parameter = {
    "algo__n_neighbors": range(1, 51, 2),
    "algo__weights": ["uniform","distance"],
    "algo__p": [1, 2]
}

model = GridSearchCV(pipeline, parameter, cv=3, scoring='r2', n_jobs=-1, verbose=1)
model.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   40.0s finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['MONTH']),
                                                                        ('categoric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                           

In [246]:
model.best_params_

{'algo__n_neighbors': 49, 'algo__p': 2, 'algo__weights': 'uniform'}

In [247]:
model.score(X_train, y_train), model.score(X_test, y_test)

(0.032572447019895256, -0.003034463790977604)

# Membandingkan Model terbaik

RandomForest = 0.018585032412645086<br>
SVM = -0.08794601517976242<br>
KNN = -0.003034463790977604<br>

Model yang memiliki score terbaik adalah RandomForest, maka model tersebut akan digunakan untuk melakukan prediksi

# Prediksi

In [255]:
gsp.rf_params

{'algo__n_estimators': [100, 150, 200],
 'algo__max_depth': [20, 50, 80],
 'algo__max_features': [0.3, 0.6, 0.8],
 'algo__min_samples_leaf': [1, 5, 10]}

In [256]:
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(), ['MONTH']),
    ('categoric', cat_pipe(encoder='onehot'), ['DAY_OF_WEEK', 'DISTRICT', 'offense_description']),
    
])


pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', RandomForestRegressor(n_jobs=-1, random_state=42))
])


model = GridSearchCV(pipeline, gsp.rf_params, scoring = 'r2', cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

print(model.best_params_)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:  4.2min finished


{'algo__max_depth': 20, 'algo__max_features': 0.3, 'algo__min_samples_leaf': 10, 'algo__n_estimators': 200}
0.08713449086717528 0.0065299143774378106 0.018585032412645086


In [257]:
X_train.iloc[:1]

,DISTRICT,MONTH,DAY_OF_WEEK,offense_description
2441,E13,12,Sunday,towed motor vehicle


In [258]:
X_pred = pd.DataFrame([['D14', 6, 'Thursday', 'vandalism']], columns=X_train.columns) 

In [259]:
model.predict(X_pred)

array([12.58779809])